In [14]:
import numpy as np
from IPython.display import HTML, display
from pydrake.all import (
    AbstractValue, 
    ConstantVectorSource,
    DiagramBuilder,
    LeafSystem,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    PiecewisePose,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    TrajectorySource,
    AddMultibodyPlantSceneGraph,
    Parser,
)

In [2]:
from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad

In [22]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [4]:
class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DecalreAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPosee
        )
        
    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))

In [20]:
def MakeJacoStation(
    model_directives=None,
    filename=None,
    time_step=0.002,
):
    """
    Creates a kinova jaco station system, which is a sub-diagram containing:
     - A MultibodyPlant with populated via the Parser from the
       `model_directives` argument AND the `filename` argument.
     - A SceneGraph
     - For each model instance starting with ``
     
    Args:
        model_directives: a string containing any model directives to be parsed
        
        filename: a string containing the name of an sdf, urdf, mujoco xml. or
        model directives yaml file.
        
        time_step: the standard MultibodyPlant time step.

    """
    builder = DiagramBuilder()
    
    # add kinova jaco arm
    plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=time_step
    )
    parser = Parser(plant)
    if model_directives:
        parser.AddModelsFromString(model_directives, ".dmd.yaml")
    if filename:
        parser.AddModelsFromUrl(filename)
    plant.Finalize()
    
    # Export "cheat" ports.
    builder.ExportOutput(scene_graph.get_query_output_port(), "query_object")
    
    diagram = builder.Build()
    diagram.set_name("JacoStation")
    return diagram

In [39]:
class Jaco_Painter:
    def __init__(self, traj=None):
        builder = DiagramBuilder()
        # set up the systems of jaco station
        self.station = MakeJacoStation(
            filename="package://drake/manipulation/models/jaco_description/urdf/j2s7s300_sphere_collision.urdf")
        
        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")
        """
        controller_plant = self.station.GetSubsystemByName(
            "jaco_controller"
        ).get_multibody_plant_for_control()
        """
        
        # add trajectory source
        if traj is not None:
            traj_source = builder.AddSystem(PoseTrajectorySource(traj))
            # TODO:
            # add controller
        
        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder,
            self.station.GetOutputPort("query_object"),
            meshcat,
            params
        )
        
        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName("j2s7s300_end_effector")
        self.world_frame = self.plant.world_frame()
        
        context = self.CreateDefaultContext()
        self.diagram.ForcedPublish(context)
        
    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )
        
    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context
        )
        station_context = self.diagram.GetMutableSubsystemContext(
            self.station, context
        )
        
        # provide initial states
        q0 = np.array(
            [
                1.8,
                3.44,
                3.14,
                0.76,
                4.63,
                4.49,
                5.03,
            ]
        )
        # set the joint positions of the jaco arm
        jaco = self.plant.GetModelInstanceByName("j2s7s300")
        self.plant.SetPositions(plant_context, jaco, q0)
        # TODO:
        return context
    
    def get_X_WG(self, context=None):
        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
            plant_context, frame_A=self.world_frame, frame_B=self.gripper_frame
        )
        return X_WG
        
    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)
        simulator.set_target_realtime_rate(1.0)
        
        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdcanveTo(duration)

In [40]:
painter = Jaco_Painter()

SystemExit: Failure at bazel-out/k8-opt/bin/tools/install/libdrake/_virtual_includes/drake_shared_library/drake/multibody/plant/multibody_plant.h:2152 in SetPositions(): condition 'q_instance.size() == num_positions(model_instance)' failed.

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [41]:
X_WG = painter.get_X_WG()
painter.visualize_frame("gripper_current", X_WG)